### This time start without sampling and test with LSTM

Without sampling usually failed bz it is unbalanced class dataset and you need to oversampling and undersampling.

Decision Tree ( av-precison = 0.258728) without sampling  
DNN-LSTM ( av-precison = 0.25872) without sampling

In [ ]:
import os 
import numpy as np 
import pandas as pd

In [ ]:
train_feature = np.load('Data/mix/train_feature.npy')
valid_feature = np.load('Data/mix/valid_feature.npy')
test_feature = np.load('Data/mix/test_feature.npy')
train_label = np.load('Data/mix/train_label.npy')
valid_label = np.load('Data/mix/valid_label.npy')
test_label = np.load('Data/mix/test_label.npy')

In [ ]:
train_feature.shape, train_label.shape, test_feature.shape, test_label.shape, valid_feature.shape, valid_label.shape

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import classification_report

def eval_metrics(classifier, test_features, test_labels):
    
    # make prediction
    predictions   = classifier.predict(test_features)
    
    base_score   = classifier.score(test_features, test_labels)
    accuracy = accuracy_score(test_labels, predictions)
    av_precision = average_precision_score(test_labels, predictions)
    
    target_names = ['StartHesitation','Turn','Walking', 'All_zero']
    print("Classification report")
    print("---------------------","\n")
    print(classification_report(test_labels, predictions, target_names=target_names),"\n")

    print("Accuracy Measures")
    print("---------------------","\n")
    print("Base score: ", base_score)
    print("Accuracy: ", accuracy)
    print("Avarge Precision: ", av_precision)
    
    return base_score,accuracy,av_precision

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
Models = {                         
    "Decision Tree": DecisionTreeClassifier(),      
    "KNearest": KNeighborsClassifier(n_jobs=-1),           
    }

In [ ]:
import mlflow
import mlflow.tensorflow
mlflow.set_tracking_uri("http://127.0.0.1:5000")
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

In [ ]:
counter = 1
for Model_Name, classifier in Models.items(): 
    # with mlflow.start_run(nested=True):
    print(f"{counter}. {Model_Name}")
    
    with mlflow.start_run():
        # fit the model
        from joblib import parallel_backend
        with parallel_backend('threading', n_jobs=-1):
            classifier.fit(train_feature, train_label)
        counter = counter + 1
        
        # Calculate the metrics
        base_score,accuracy,av_precision = eval_metrics(classifier,
                                                        valid_feature,
                                                        valid_label)  
        
        mlflow.log_param("Model"           , Model_Name)
        mlflow.log_param("Dataset" , "Mix")
        mlflow.log_metric("base_score"     , base_score)
        mlflow.log_metric("accuracy"       , accuracy)
        mlflow.log_metric("av_precision"   , av_precision)
        
        if av_precision > 0.95 :
            mlflow.sklearn.log_model(classifier,Model_Name, signature=signature)
            print(f"f1 socre is more than 0.945 so the {Model_Name} is saved")
        else :
            print(f"Because f1 socre is not quality. The model is skip to saving phase.")
        
        print("________________________________________")

### Tensorflow

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
from datetime import datetime

In [ ]:
epoch = 5
input_shape = 3

inputs = layers.Input(shape=input_shape, name="input_layer")
x = layers.Lambda(lambda x: tf.expand_dims(x, axis=-1))(inputs)
x = layers.LSTM(64)(x) # return vector for whole sequence
x = layers.Dense(16, activation="relu")(x) 
outputs = layers.Dense(4, activation="softmax")(x)
model = tf.keras.Model(inputs, outputs, name="model_LSTM")



model.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

model.summary()

In [ ]:
# Callbacks
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", # watch the val loss metric
                                                  patience=5) # if val loss decreases for 3 epochs in a row, stop training

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",  
                                                 factor=0.2, # multiply the learning rate by 0.2 (reduce by 5x)
                                                 patience=3,
                                                 verbose=1, # print out when learning rate goes down 
                                                 min_lr=1e-7)

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_feature, train_label))
train_dataset =  train_dataset.batch(2048).prefetch(tf.data.AUTOTUNE)

valid_dataset = tf.data.Dataset.from_tensor_slices((valid_feature, valid_label))
valid_dataset = valid_dataset.batch(2048).prefetch(tf.data.AUTOTUNE)

test_dataset = tf.data.Dataset.from_tensor_slices((test_feature,test_label))
test_dataset = test_dataset.batch(2048).prefetch(tf.data.AUTOTUNE)

train_dataset, valid_dataset, test_dataset

In [ ]:
# fit the model
start = datetime.now()
with mlflow.start_run():
    mlflow.tensorflow.autolog()
    
    # fit the model
    # with tf.device('/GPU:0'):
    start = datetime.now()
    history_model = model.fit(train_dataset,
                            batch_size=2048,
                            steps_per_epoch=len(train_dataset),
                            validation_data=valid_dataset,
                            validation_steps=int(len(valid_dataset)),
                            callbacks=[early_stopping, reduce_lr],
                            epochs=epoch) 
    end = datetime.now()
    print(f"The time taken to train the model is {end - start}")
        
    # Evaluate model
    model.evaluate(test_dataset)
    
    # Calculate the metrics
    # model_results = eval_metrics(model, 
    #                              test_features = test_feature,
    #                              test_labels = test_label)
    
    mlflow.log_param("Model"   , "LSTM")
    mlflow.log_param("Dataset" , "Mix")
    # mlflow.log_params(model_results)
    mlflow.tensorflow.autolog()

print("________________________________________")

In [ ]:

# Calculate the metrics
model_preds_probs   = model.predict(test_feature)
av_precision    = average_precision_score(test_label, model_preds_probs)
av_precision

#Mix DNN model

In [ ]:
# Build the model 
epoch = 10
input_shape = 3

inputs = layers.Input(shape=input_shape, name="input_layer")
x = layers.Dense(32, activation ='relu')(inputs)
x = layers.Dense(64, activation = 'relu')(x)
# x = layers.BatchNormalization()(x)
# x = layers.Dropout(0.1)(x)
# x = layers.Dense(128, activation ='relu')(x)
# x = layers.Dense(512, activation = 'relu')(x)
# x = layers.BatchNormalization()(x)
# x = layers.Dropout(0.25)(x)
# x = layers.Dense(1024, activation ='relu')(x)
# x = layers.Dense(512, activation = 'relu')(x)
# x = layers.BatchNormalization()(x)
# x = layers.Dropout(0.5)(x)
# x = layers.Dense(128, activation="relu")(x)
# x = layers.Dense(64, activation="relu")(x)
x = layers.Dense(32, activation="relu")(x)
outputs = layers.Dense(4, activation="softmax",name="output_layer")(x)      
model_DNN = tf.keras.Model(inputs, outputs) 

model_DNN.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

model_DNN.summary()



In [ ]:
from sklearn.model_selection import cross_val_score
counter = 1
Model_Name = 'DNN_model'
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.get_logger().setLevel('ERROR')
tf.autograph.set_verbosity(1)
tf.set_seed = 42
np.random.seed(40)

with mlflow.start_run():
    mlflow.tensorflow.autolog()
    
    # fit the model
    # with tf.device('/GPU:0'):
    start = datetime.now()
    history_model = model_DNN.fit(train_dataset,
                                  batch_size=2048,
                                  steps_per_epoch=len(train_dataset),
                                  validation_data=valid_dataset,
                                  validation_steps=int(len(valid_dataset)),
                                  callbacks=[early_stopping, reduce_lr],
                                  epochs=epoch) 
    end = datetime.now()
    print(f"The time taken to train the model is {end - start}")
        
    # Evaluate model
    model.evaluate(test_dataset)
    
    # Calculate the metrics
    # model_results = eval_metrics(model, 
    #                              test_features = test_feature,
    #                              test_labels = test_label)
    
    mlflow.log_param("Model"           , Model_Name)
    mlflow.log_param("Dataset" , "Defog")
    # mlflow.log_params(model_results)
    mlflow.tensorflow.autolog()
    
    
    print("________________________________________")

In [ ]:
# Calculate the metrics
model_preds_probs   = model.predict(test_feature)
model_accuracy = accuracy_score(test_label, model_preds_probs) 
print(model_accuracy)
av_precision    = average_precision_score(test_label, model_preds_probs)
print(av_precision)